In [60]:
import pandas as pd
import numpy as np

For TGOS data

In [61]:
file_list = [14, 15, 16, 17, 18, 21, 22, 23, 29, 30, 31]

In [62]:
for f in file_list:
    path = 'addr_' + str(f) + '.csv'
    print(path)
    df = pd.read_csv('addr_finish/'+path, on_bad_lines='skip')
    df = df.drop(columns=['Response_Address'])
    for i in range(len(df)):
        if(';' in str(df['Response_X'].iloc[i]) or ';' in str(df['Response_Y'].iloc[i])):   # 去掉回傳筆數大於1的
            df['Response_X'].iloc[i] = np.nan
        if(df['Response_X'].iloc[i] == '-'):
            df['Response_X'].iloc[i] = np.nan
    df = df.dropna()

    
    print(len(df))
    df.to_csv('final_addr_table/'+path, index=False)
    print('-'*80)

addr_14.csv


C:\Users\user\AppData\Local\Temp\ipykernel_5220\4170425425.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Response_X'].iloc[i] = np.nan


7973
--------------------------------------------------------------------------------
addr_15.csv
8445
--------------------------------------------------------------------------------
addr_16.csv
8573
--------------------------------------------------------------------------------
addr_17.csv
8545
--------------------------------------------------------------------------------
addr_18.csv
8609
--------------------------------------------------------------------------------
addr_21.csv
8507
--------------------------------------------------------------------------------
addr_22.csv
8511
--------------------------------------------------------------------------------
addr_23.csv
8579
--------------------------------------------------------------------------------
addr_29.csv
10000
--------------------------------------------------------------------------------
addr_30.csv
10000
--------------------------------------------------------------------------------
addr_31.csv
9989
-------------

---

For Excel轉地址

In [63]:
file_list = ['00', '01', '02', '03', '19', '20', '27']
for i in file_list:
    path = 'addr_' + i + '.xlsx'
    print(path)
    output_path = 'addr_' + i + '.csv'
    df = pd.read_excel('addr_finish/'+path)
    df = pd.DataFrame.from_dict({'id': df['id'], 'Address': df['Address'], 'Response_X':df['Longitude'], 'Response_Y': df['Latitude']})
    for i in range(len(df)):
        if(df['Response_X'].iloc[i] == '-'):
            df['Response_X'].iloc[i] = np.nan
    df = df.dropna()
    print(len(df))
    print('-'*50)
    df.to_csv('final_addr_table/'+output_path, index=False)

addr_00.xlsx
9921
--------------------------------------------------
addr_01.xlsx


C:\Users\user\AppData\Local\Temp\ipykernel_5220\847845976.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Response_X'].iloc[i] = np.nan


9998
--------------------------------------------------
addr_02.xlsx
10000
--------------------------------------------------
addr_03.xlsx
1701
--------------------------------------------------
addr_19.xlsx
10000
--------------------------------------------------
addr_20.xlsx
10000
--------------------------------------------------
addr_27.xlsx
10000
--------------------------------------------------


---
合併

In [64]:
final_table = pd.DataFrame(columns=['id', 'Address', 'Response_X', 'Response_Y'])

In [65]:
file_list = ['00', '01', '02', '03', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '27', '29', '30', '31']
for i in file_list:
    path = 'final_addr_table/addr_' + i + '.csv'
    print(path)
    df = pd.read_csv(path)
    final_table = pd.concat([final_table, df])
print(len(final_table))

final_addr_table/addr_00.csv
final_addr_table/addr_01.csv
final_addr_table/addr_02.csv
final_addr_table/addr_03.csv
final_addr_table/addr_14.csv
final_addr_table/addr_15.csv
final_addr_table/addr_16.csv
final_addr_table/addr_17.csv
final_addr_table/addr_18.csv
final_addr_table/addr_19.csv
final_addr_table/addr_20.csv
final_addr_table/addr_21.csv
final_addr_table/addr_22.csv
final_addr_table/addr_23.csv
final_addr_table/addr_27.csv
final_addr_table/addr_29.csv
final_addr_table/addr_30.csv
final_addr_table/addr_31.csv
159351


In [66]:
final_table.head()

,id,Address,Response_X,Response_Y
0,1,台中市霧峰區台中縣霧峰鄉中正路969號413,120.698724,24.065998
1,2,台北市松山區敦化北路260號2樓,121.550139,25.061021
2,3,高雄市鼓山區美術東六路188號15樓,120.289846,22.663962
3,4,新北市三重區三和路4段252號2樓,121.485997,25.076812
4,5,新北市汐止區忠孝東路424號2樓,121.666064,25.071837


In [67]:
l = final_table['Address'].tolist()
for i in range(len(l)):
    l[i] = l[i].replace(',', ' ')
    l[i] = l[i].replace('，', ' ')
    l[i] = l[i].replace('\r', ' ')
    l[i] = l[i].replace('\n', ' ')
final_table['Address'] = l

In [68]:
id = final_table['id'].tolist()
for i in range(len(id)):
    id[i] = int(id[i])
final_table['id'] = id

In [69]:
final_table.to_csv('combined_addr.csv', index=False)

---

In [70]:
import psycopg2

# change these to your setting
database = 'final_project'
user = 'postgres'
password = '1234'
host = '127.0.0.1'
port = '5432'
conn = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)
with(conn):
    print("Opened database successfully")

Opened database successfully


In [57]:
# address
print('import address')

cur = conn.cursor()
cur.execute('''CREATE TABLE address
       (ID            int PRIMARY KEY  NOT NULL,
       Address        varchar,
       Response_X     float8,
       Response_Y     float8);''')
conn.commit()
print("Table created successfully")

import address
Table created successfully


In [71]:

cur = conn.cursor()
path = 'combined_addr.csv'
with open(path, 'r', encoding='utf-8') as f:
    next(f)
    cur.copy_from(f, 'address', sep=',', )
conn.commit()
print("data inserted successfully\n")

cur.execute('''ALTER TABLE address ADD geo geography;''')
conn.commit()

data inserted successfully



In [72]:
cur.execute('''update address
set geo = st_geogfromtext('POINT(' || response_x || ' ' || response_y || ')')
where geo is NULL;''')

conn.commit()
print("update geography data successfully")

update geography data successfully
